# Sistemas especialistas

In [23]:
import experta

base_conhecimento = {
    "sol": {
        "quente": "Ir à Praia",
        "ameno": "Fazer uma caminhada no parque"
    },
    "nublado": {
        "quente": "Visitar um parente com ar condicionado",
        "ameno": "Ficar em casa e tomar café"
    },
    "chuva": {
        "quente": "Ficar na chuva",
        "ameno": "Ficar em casa comendo pipoca"
    }
}

## Motor de inferência (SIMPLES)

In [24]:
def motor_inferencia_clima(fatos):
    clima = fatos.get('clima')
    temperatura = fatos.get('temperatura')

    if clima in base_conhecimento and temperatura in base_conhecimento[clima]:
        return base_conhecimento[clima][temperatura]
    else:
        return "Não tenho recomendação para esta combinação de clima e temperatura."

## Simulação

In [25]:
fatos = {
    "clima": "chuva",
    "temperatura": "ameno"
}

conclusao = motor_inferencia_clima(fatos)
print(f"Fatos: {fatos}")
print(f"Recomendação do sistema: {conclusao}")

Fatos: {'clima': 'chuva', 'temperatura': 'ameno'}
Recomendação do sistema: Ficar em casa comendo pipoca


# Forward e Backward

In [26]:
fatos = ["tem_pelos", "voa", "produz_leite"]
regras = [
    {
        "se": ["tem_penas", "voa"],
        "entao": "e_pessaro"
    },
    {
        "se": ["tem_penas", "pode_cantar"],
        "entao": "e_canario"
    },
    {
        "se": ["tem_pelos", "produz_leite"],
        "entao": "e_mamifero"
    },
    {
        "se": ["tem_pelos", "voa"],
        "entao": "e_morcego"
    },
    {
        "se": ["tem_pelos", "pode_cantar"],
        "entao": "e_pirata"
    },
    {
        "se": ["tem_pelos", "late"],
        "entao": "e_cachorro"
    }
]

## Motor de Inferência (Forward)

In [27]:
def motor_inferencia_forward(fatos_init, regras):
    fatos_derivados = list(fatos_init)
    novo_fato = True

    while novo_fato:
        novo_fato = False
        for regra in regras:
            condicao_satisfeita = all(condicao in fatos_derivados for condicao in regra["se"])

            if condicao_satisfeita and regra["entao"] not in fatos_derivados:
                fatos_derivados.append(regra["entao"])
                print(f"Regra disparada: SE {regra['se']} ENTAO {regra['entao']}")
                print(f"Fatos adicionado: {regra['entao']}")
                novo_fato = True

    return fatos_derivados

## Simulação

In [28]:
print(f"Fatos iniciais: {fatos}")
fatos_finais = motor_inferencia_forward(fatos, regras)
print(f"Fatos finais: {fatos_finais}")

Fatos iniciais: ['tem_pelos', 'voa', 'produz_leite']
Regra disparada: SE ['tem_pelos', 'produz_leite'] ENTAO e_mamifero
Fatos adicionado: e_mamifero
Regra disparada: SE ['tem_pelos', 'voa'] ENTAO e_morcego
Fatos adicionado: e_morcego
Fatos finais: ['tem_pelos', 'voa', 'produz_leite', 'e_mamifero', 'e_morcego']


# Biblioteca Experta


In [1]:
from experta import *

class Caracteristica(Fact):
    "Representa uma característica observada"
    pass

class Animal(KnowledgeEngine):
    @DefFacts()
    def fatos_iniciais(self):
        yield Caracteristica("tem_penas")
        yield Caracteristica("voa")
        yield Caracteristica("pode_cantar")
        print("Fatos iniciais carregados.")

    @Rule(Caracteristica("tem_penas"), Caracteristica("voa"))
    def regra_e_passaro(self):
        print("Regra disparada: SE tem_penas e voa ENTAO e_passaro")
        self.declare(Fact(animal="e_passaro"))

    @Rule(Fact(animal="e_passaro"), Caracteristica("pode_cantar"))
    def regra_e_canario(self):
        print("Regra disparada: SE e_passaro e pode_cantar ENTAO e_canario")
        self.declare(Fact(animal="canário"))

    @Rule(Fact(animal=MATCH.tipo))
    def print_resultado(self, tipo):
        if tipo == "canário":
            print(f"Conclusão final do sistema, o tipo de anial é: {tipo}")

animal = Animal()
animal.reset()
animal.run()


Fatos iniciais carregados.
Regra disparada: SE tem_penas e voa ENTAO e_passaro
Regra disparada: SE e_passaro e pode_cantar ENTAO e_canario
Conclusão final do sistema, o tipo de anial é: canário


In [1]:
from experta import *

class Sintoma(Fact):
    pass

class SistemaDiagnosticoPlanta(KnowledgeEngine):
    @DefFacts()
    def sintomas_observados(self):
        yield Sintoma(tipo="folhas", estado="amareladas")
        yield Sintoma(tipo="solo", estado="muito umido")
        yield Sintoma(tipo="solo", estado="seco")
        yield Sintoma(tipo="folhas", estado="murchas")
        yield Sintoma(tipo="pragas", estado="pontos brancos")

    @Rule(Sintoma(tipo="folhas", estado="amareladas"), Sintoma(tipo="solo", estado="muito umido"))
    def regra_1(self):
        print("Regra disparada: SE folhas e amareladas e solo e muito umido ENTAO Excesso de água")
        self.declare(Fact(diagnostico="Excesso de água", solucao="Reduza a frequência de irrigação as plantas"))

    @Rule(Sintoma(tipo="folhas", estado="amareladas"), Sintoma(tipo="solo", estado="seco"))
    def regra_2(self):
        print("Regra disparada: SE folhas e amareladas e solo e seco ENTAO Falta de água")
        self.declare(Fact(diagnostico="Falta de água", solucao="Aumente a frequência de irrigação as plantas"))

    #@Rule(Sintoma(tipo="folhas", estado="murchas"), Sintoma(tipo="solo", estado="muito umido"))

    @Rule(Sintoma(tipo="pragas", estado="pontos brancos"))
    def regra_4(self):
        print("Regra disparada: SE folhas tem praga e pontos brancos ENTAO As plantas estão doentes")
        self.declare(Fact(diagnostico="As plantas estão doentes", solucao="Limpe as plantas com pano umido"))

    @Rule(Fact(diagnostico=MATCH.diagnostico, solucao=MATCH.solucao))
    def resultados(self, diagnostico, solucao):
        print("------------------------------------")
        print(f"Problema indentificado: {diagnostico}")
        print(f"Solução para o problema: {solucao}")
        print("------------------------------------")

def interface_usuario(engine):
    print("Por favor, informe os sintomas do problema:")

    #pergunta 1
    resposta = input("As folhas da plantao estão amareladas? (s/n) ")
    if resposta == "s":
        engine.declare(Sintoma(tipo="folhas", estado="amareladas"))

        resposta_solo = input("O solo parece muito umido? (s/n) ")
        if resposta_solo == "s":
            engine.declare(Sintoma(tipo="solo", estado="muito umido"))
        else:
            engine.declare(Sintoma(tipo="solo", estado="seco"))

    #pergunta 2
    resposta = input("Você vê pequenos pontos brancos nas folhas? (s/n) ")
    if resposta == "s":
        engine.declare(Sintoma(tipo="pragas", estado="pontos brancos"))

engine = SistemaDiagnosticoPlanta()
engine.reset()
interface_usuario(engine)
engine.run()

Por favor, informe os sintomas do problema:
Regra disparada: SE folhas tem praga e pontos brancos ENTAO As plantas estão doentes
------------------------------------
Problema indentificado: As plantas estão doentes
Solução para o problema: Limpe as plantas com pano umido
------------------------------------
Regra disparada: SE folhas e amareladas e solo e seco ENTAO Falta de água
------------------------------------
Problema indentificado: Falta de água
Solução para o problema: Aumente a frequência de irrigação as plantas
------------------------------------
Regra disparada: SE folhas e amareladas e solo e muito umido ENTAO Excesso de água
------------------------------------
Problema indentificado: Excesso de água
Solução para o problema: Reduza a frequência de irrigação as plantas
------------------------------------
